In [1]:
import pandas as pd
import arff as liacarff

# 加载 ARFF 文件
with open('PROMISE_DOSSPRE_OLD_3.arff', 'r') as f:
    data_dict = liacarff.load(f)

# 提取数据
data = data_dict['data']

# 转换为 DataFrame
df = pd.DataFrame(data, columns=[attr[0] for attr in data_dict['attributes']])
df['ProjectID'] = df['ProjectID'].astype(int)  # 将 ProjectID 列转换为整数类型

# 显示 DataFrame
print(df)

      ProjectID                                    RequirementText _class_
0             1  The system shall refresh the display every 60 ...      PE
1             1  The application shall match the color of the s...      LF
2             1  If projected  the data must be readable.  On a...      US
3             1  The product shall be available during normal b...       A
4             1  If projected  the data must be understandable....      US
...         ...                                                ...     ...
2052          9  The database may trade off fidelity through ca...      FT
2053          9  The API shall have master topology replicating...      FT
2054          9  The system must parse, filter, transform and s...      FT
2055          9  The application shall employ real-user monitor...      FT
2056          9  The software should apply graceful degradation...      FT

[2057 rows x 3 columns]


In [2]:
# DATA PREPROCESSING
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re
import string

# 下载NLTK的停用词和词性标注器数据
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# lower
df['RequirementText'] = df['RequirementText'].str.lower()

# # Remove punctuation, leading and trailing spaces
# df['RequirementText'] = df['RequirementText'].apply(lambda x: re.sub(f"[{re.escape(string.punctuation)}]", "", x.strip()))

# # 分词
df['RequirementText'] = df['RequirementText'].apply(word_tokenize)

# # stopwords
# stop_words = set(stopwords.words('english'))
# df['RequirementText'] = df['RequirementText'].apply(lambda x: [word for word in x if word not in stop_words])

# # stemming
stemmer = PorterStemmer()
df['RequirementText'] = df['RequirementText'].apply(lambda x: [stemmer.stem(word) for word in x])

# # lemmatization
# lemmatizer = WordNetLemmatizer()
# df['RequirementText'] = df['RequirementText'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

# # 合并词语为字符串
df['RequirementText'] = df['RequirementText'].apply(lambda x: ' '.join(x))

# Replace multiple spaces with a single space
df['RequirementText'] = df['RequirementText'].apply(lambda x: re.sub(' +', ' ', x))

# 移除整个列中每个字符串首尾的全部空格
df['RequirementText'] = df['RequirementText'].str.strip()

# 查看处理后的数据集
print(df.head())

[nltk_data] Downloading package stopwords to /home/li/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/li/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/li/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


   ProjectID                                    RequirementText _class_
0          1  the system shall refresh the display everi 60 ...      PE
1          1  the applic shall match the color of the schema...      LF
2          1  if project the data must be readabl . on a 10x...      US
3          1  the product shall be avail dure normal busi ho...       A
4          1  if project the data must be understand . on a ...      US


In [3]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

In [4]:
from sklearn.model_selection import train_test_split
y = df['_class_']
X = df['RequirementText']

# 假设 X 是特征，y 是目标变量（类别标签）
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y)

In [6]:
# 使用TfidfVectorizer进行转换
ngram_range = (1,2)
min_df = 0.01 # ignore terms that appear in less than 1% of the documents
max_df = 0.8 # ignore terms that appear in more than 80% of the documents

tfidf_vectorizer = CountVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df)

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

model_tfidf = MultinomialNB()
model_tfidf.fit(X_train_tfidf, y_train)
y_pred_tfidf = model_tfidf.predict(X_test_tfidf)

# 评估模型性能
print("Accuracy (TF-IDF):", accuracy_score(y_test, y_pred_tfidf))
print("Classification Report (TF-IDF):\n", classification_report(y_test, y_pred_tfidf))

Accuracy (TF-IDF): 0.7038834951456311
Classification Report (TF-IDF):
               precision    recall  f1-score   support

           A       0.67      0.67      0.67        12
           F       0.76      0.80      0.78        44
          FT       0.56      0.71      0.63         7
           L       0.67      0.44      0.53         9
          LF       0.80      0.50      0.62        16
          MN       0.67      0.50      0.57        16
           O       0.85      0.85      0.85        13
          PE       0.71      0.68      0.69        25
          PO       0.64      0.88      0.74         8
          SC       0.69      0.90      0.78        10
          SE       0.68      0.79      0.73        24
          US       0.64      0.64      0.64        22

    accuracy                           0.70       206
   macro avg       0.69      0.70      0.68       206
weighted avg       0.71      0.70      0.70       206



In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

# 创建一个Pipeline，仅包括Multinomial Naive Bayes模型
pipeline = Pipeline([
    ('clf', MultinomialNB())
])

# 设置需要调优的参数，包括TF-IDF的参数
parameters = {
    'clf__alpha': (1e-2, 1e-3)
}

# 使用GridSearchCV进行模型选择和调参，设置评价标准为f1 score
grid_search = GridSearchCV(pipeline, parameters, scoring='f1_weighted', cv=5, refit=True)
grid_search.fit(X_train_tfidf, y_train)

# 输出最佳参数和对应的f1 score
print("Best parameters found: ", grid_search.best_params_)
print("Best f1 score: ", grid_search.best_score_)

best_model = grid_search.best_estimator_

y_pred_tuned = best_model.predict(X_test_tfidf)

# 输出准确率和分类报告
print("Accuracy (Tuned):", accuracy_score(y_test, y_pred_tuned))
print("Classification Report (Tuned):\n", classification_report(y_test, y_pred_tuned))

Best parameters found:  {'clf__alpha': 0.01}
Best f1 score:  0.6518908560920208
Accuracy (Tuned): 0.6844660194174758
Classification Report (Tuned):
               precision    recall  f1-score   support

           A       0.64      0.58      0.61        12
           F       0.77      0.75      0.76        44
          FT       0.62      0.71      0.67         7
           L       0.57      0.44      0.50         9
          LF       0.92      0.69      0.79        16
          MN       0.67      0.50      0.57        16
           O       0.79      0.85      0.81        13
          PE       0.61      0.68      0.64        25
          PO       0.62      0.62      0.62         8
          SC       0.69      0.90      0.78        10
          SE       0.61      0.71      0.65        24
          US       0.64      0.64      0.64        22

    accuracy                           0.68       206
   macro avg       0.68      0.67      0.67       206
weighted avg       0.69      0.68      

In [7]:
# F binary

df['binary_label'] = df['_class_'].apply(lambda x: 1 if x == 'F' else 0)
X_train, X_test, y_train, y_test = train_test_split(df['RequirementText'], df['binary_label'], test_size=0.2, random_state=42)

# 使用TfidfVectorizer进行转换
ngram_range = (1,2)
min_df = 0.01 # ignore terms that appear in less than 1% of the documents
max_df = 0.8 # ignore terms that appear in more than 80% of the documents

tfidf_vectorizer = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        norm='l2',
                        sublinear_tf=True)

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

model_tfidf = MultinomialNB()
model_tfidf.fit(X_train_tfidf, y_train)

y_pred_tfidf = model_tfidf.predict(X_test_tfidf)

print("Accuracy (TF-IDF):", accuracy_score(y_test, y_pred_tfidf))
print("Classification Report (TF-IDF):\n", classification_report(y_test, y_pred_tfidf))

Accuracy (TF-IDF): 0.8907766990291263
Classification Report (TF-IDF):
               precision    recall  f1-score   support

           0       0.90      0.97      0.93       324
           1       0.84      0.60      0.70        88

    accuracy                           0.89       412
   macro avg       0.87      0.79      0.82       412
weighted avg       0.89      0.89      0.88       412



In [8]:
filtered_df = df[df['_class_'] != 'F']
X_train, X_test, y_train, y_test = train_test_split(filtered_df['RequirementText'], filtered_df['_class_'], test_size=0.2, random_state=42, stratify=filtered_df['_class_'])

# 使用TfidfVectorizer进行转换
ngram_range = (1,2)
min_df = 0.01 # ignore terms that appear in less than 1% of the documents
max_df = 0.8 # ignore terms that appear in more than 80% of the documents

tfidf_vectorizer = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        norm='l2',
                        sublinear_tf=True)

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

model_tfidf = MultinomialNB()
model_tfidf.fit(X_train_tfidf, y_train)

y_pred_tfidf = model_tfidf.predict(X_test_tfidf)

print("Accuracy (TF-IDF):", accuracy_score(y_test, y_pred_tfidf))
print("Classification Report (TF-IDF):\n", classification_report(y_test, y_pred_tfidf, zero_division=1))

Accuracy (TF-IDF): 0.718266253869969
Classification Report (TF-IDF):
               precision    recall  f1-score   support

           A       0.89      0.71      0.79        24
          FT       0.91      0.71      0.80        14
           L       0.67      0.12      0.20        17
          LF       0.79      0.70      0.74        33
          MN       0.79      0.70      0.74        33
           O       0.69      0.77      0.73        26
          PE       0.70      0.80      0.75        50
          PO       0.88      0.44      0.58        16
          SC       0.94      0.79      0.86        19
          SE       0.57      0.91      0.70        47
          US       0.70      0.73      0.71        44

    accuracy                           0.72       323
   macro avg       0.77      0.67      0.69       323
weighted avg       0.74      0.72      0.71       323



In [9]:
filtered_df = df[df['_class_'].isin(['MN', 'SC', 'PE'])]
X_train, X_test, y_train, y_test = train_test_split(filtered_df['RequirementText'], filtered_df['_class_'], test_size=0.2, random_state=42, stratify=filtered_df['_class_'])

# 使用TfidfVectorizer进行转换
ngram_range = (1,2)
min_df = 0.01 # ignore terms that appear in less than 1% of the documents
max_df = 0.8 # ignore terms that appear in more than 80% of the documents

tfidf_vectorizer = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        norm='l2',
                        sublinear_tf=True)

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

model_tfidf = MultinomialNB()
model_tfidf.fit(X_train_tfidf, y_train)

y_pred_tfidf = model_tfidf.predict(X_test_tfidf)

print("Accuracy (TF-IDF):", accuracy_score(y_test, y_pred_tfidf))
print("Classification Report (TF-IDF):\n", classification_report(y_test, y_pred_tfidf, zero_division=1))

Accuracy (TF-IDF): 0.8725490196078431
Classification Report (TF-IDF):
               precision    recall  f1-score   support

          MN       0.90      0.79      0.84        33
          PE       0.83      0.98      0.90        50
          SC       1.00      0.74      0.85        19

    accuracy                           0.87       102
   macro avg       0.91      0.83      0.86       102
weighted avg       0.88      0.87      0.87       102



In [10]:
filtered_df = df[df['_class_'].isin(['A', 'FT'])]
X_train, X_test, y_train, y_test = train_test_split(filtered_df['RequirementText'], filtered_df['_class_'], test_size=0.2, random_state=42, stratify=filtered_df['_class_'])

# 使用TfidfVectorizer进行转换
ngram_range = (1,2)
min_df = 0.01 # ignore terms that appear in less than 1% of the documents
max_df = 0.8 # ignore terms that appear in more than 80% of the documents

tfidf_vectorizer = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        norm='l2',
                        sublinear_tf=True)

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

model_tfidf = MultinomialNB()
model_tfidf.fit(X_train_tfidf, y_train)

y_pred_tfidf = model_tfidf.predict(X_test_tfidf)

print("Accuracy (TF-IDF):", accuracy_score(y_test, y_pred_tfidf))
print("Classification Report (TF-IDF):\n", classification_report(y_test, y_pred_tfidf, zero_division=1))

Accuracy (TF-IDF): 0.8461538461538461
Classification Report (TF-IDF):
               precision    recall  f1-score   support

           A       0.82      0.96      0.88        24
          FT       0.91      0.67      0.77        15

    accuracy                           0.85        39
   macro avg       0.87      0.81      0.83        39
weighted avg       0.86      0.85      0.84        39



In [11]:
filtered_df = df[df['_class_'].isin(['SE', 'L'])]
X_train, X_test, y_train, y_test = train_test_split(filtered_df['RequirementText'], filtered_df['_class_'], test_size=0.2, random_state=42, stratify=filtered_df['_class_'])

# 使用TfidfVectorizer进行转换
ngram_range = (1,2)
min_df = 0.01 # ignore terms that appear in less than 1% of the documents
max_df = 0.8 # ignore terms that appear in more than 80% of the documents

tfidf_vectorizer = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        norm='l2',
                        sublinear_tf=True)

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

model_tfidf = MultinomialNB()
model_tfidf.fit(X_train_tfidf, y_train)

y_pred_tfidf = model_tfidf.predict(X_test_tfidf)

print("Accuracy (TF-IDF):", accuracy_score(y_test, y_pred_tfidf))
print("Classification Report (TF-IDF):\n", classification_report(y_test, y_pred_tfidf, zero_division=1))

Accuracy (TF-IDF): 0.8307692307692308
Classification Report (TF-IDF):
               precision    recall  f1-score   support

           L       1.00      0.35      0.52        17
          SE       0.81      1.00      0.90        48

    accuracy                           0.83        65
   macro avg       0.91      0.68      0.71        65
weighted avg       0.86      0.83      0.80        65



In [12]:
filtered_df = df[df['_class_'].isin(['O', 'US'])]
X_train, X_test, y_train, y_test = train_test_split(filtered_df['RequirementText'], filtered_df['_class_'], test_size=0.2, random_state=42, stratify=filtered_df['_class_'])

# 使用TfidfVectorizer进行转换
ngram_range = (1,2)
min_df = 0.01 # ignore terms that appear in less than 1% of the documents
max_df = 0.8 # ignore terms that appear in more than 80% of the documents

tfidf_vectorizer = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        norm='l2',
                        sublinear_tf=True)

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

model_tfidf = MultinomialNB()
model_tfidf.fit(X_train_tfidf, y_train)

y_pred_tfidf = model_tfidf.predict(X_test_tfidf)

print("Accuracy (TF-IDF):", accuracy_score(y_test, y_pred_tfidf))
print("Classification Report (TF-IDF):\n", classification_report(y_test, y_pred_tfidf, zero_division=1))

Accuracy (TF-IDF): 0.9571428571428572
Classification Report (TF-IDF):
               precision    recall  f1-score   support

           O       0.96      0.92      0.94        26
          US       0.96      0.98      0.97        44

    accuracy                           0.96        70
   macro avg       0.96      0.95      0.95        70
weighted avg       0.96      0.96      0.96        70

